## install + import requirements

In [1]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import cv2
import face_recognition
import numpy as np
import speech_recognition as sr
import threading
import pyaudio
import uuid
import io
from PIL import Image

/Users/rachelchen/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import firebase_admin

cred = firebase_admin.credentials.Certificate('hw23-e0512-firebase-adminsdk-3ax9k-293086f6f4.json')

## set up firebase

In [6]:
from firebase_admin import credentials, firestore, storage
from google.cloud import storage

firebase_app = firebase_admin.initialize_app(cred,{
    'storageBucket': 'gs://hw23-e0512.appspot.com'
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [5]:
db = firestore.client()
bucket = storage.bucket('gs://hw23-e0512.appspot.com')

TypeError: 'module' object is not callable

In [5]:
rachel_image = face_recognition.load_image_file("rachel.jpg")
rachel_encoding = face_recognition.face_encodings(rachel_image)[0]

xander_image = face_recognition.load_image_file("xander.jpg")
xander_encoding = face_recognition.face_encodings(xander_image)[0]

In [6]:
known_face_encodings = [
    rachel_encoding,
    xander_encoding
]
known_face_names = [
    "Rachel Chen",
    "Xander Chin"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


In [7]:
def fetch_encodings_from_firestore():
    fetched_encodings = []
    fetched_names = []
    
    # Fetch data from Firestore
    docs = db.collection('people').stream()
    for doc in docs:
        data = doc.to_dict()
        if 'image_enc' in data and 'name' in data:
            fetched_encodings.append(np.array(data['image_enc']))
            fetched_names.append(data['name'])
    
    return fetched_encodings, fetched_names

def find_similar_face_key(face_encoding, faces_dict, tolerance=0.6):
    for face_key in faces_dict.keys():
        if np.linalg.norm(np.array(face_key) - face_encoding) < tolerance:
            return face_key
    return None

def upload_face_image(image, path):
    """Uploads the face image to Firebase Storage."""
    pil_image = Image.fromarray(image)
    byte_stream = io.BytesIO()
    pil_image.save(byte_stream, format='JPEG')
    byte_stream.seek(0)

    blob = bucket.blob(path)
    blob.upload_from_file(byte_stream, content_type='image/jpeg')
    return blob.public_url


# Dictionary to track unrecognized faces
unrecognized_faces = {}
unrecognized_threshold = 2  # Number of frames to confirm an unrecognized face

video_capture = cv2.VideoCapture(1)


In [8]:

def camera_operations(video_capture):
    face_encodings_from_db, face_names_from_db = fetch_encodings_from_firestore()
    unrecognized_faces = {}
    unrecognized_threshold = 2  # Set your threshold

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])

        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            top = max(top, 0)
            right = min(right, frame.shape[1])
            bottom = min(bottom, frame.shape[0])
            left = max(left, 0)

            face_image = frame[top:bottom, left:right]
            
            if face_image.size > 0:
                print(face_image)
                matches = face_recognition.compare_faces(face_encodings_from_db, face_encoding, tolerance=0.5)
                if True in matches:
                    first_match_index = matches.index(True)
                    name = face_names_from_db[first_match_index]
                else:
                    # Generate unique ID for the image and upload it
                    unique_id = str(uuid.uuid4())
                    image_path = f"faces/{unique_id}.jpg"
                    image_url = upload_face_image(face_image, image_path)

                    # Save the encoding and image URL in Firestore
                    doc_ref = db.collection('people').document(unique_id)
                    doc_ref.set({
                        'image_enc': face_encoding.tolist(),
                        'name': "Unnamed Person",
                        'image_url': image_url
                    })

                    # Update local encodings for real-time comparison
                    face_encodings_from_db.append(face_encoding)
                    face_names_from_db.append("Unnamed Person")

                    name = "Unnamed Person"

                face_names.append(name)

            # Drawing the results on the frame
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            cv2.imshow('Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    video_capture.release()
    cv2.destroyAllWindows()

# Example usage
video_capture = cv2.VideoCapture(0)
camera_operations(video_capture)

In [9]:
camera_operations(cv2.VideoCapture(1))

[[[ 51  46  78]
  [ 51  49  81]
  [ 60  57  89]
  ...
  [  6   6  15]
  [  6   6  15]
  [  7   7  16]]

 [[ 50  48  79]
  [ 55  53  82]
  [ 63  61  91]
  ...
  [  4   7  15]
  [  6   6  15]
  [  7   7  16]]

 [[ 53  50  82]
  [ 57  55  85]
  [ 63  61  91]
  ...
  [  3   5  14]
  [  6   6  15]
  [  6   6  15]]

 ...

 [[ 43  57  83]
  [ 45  60  85]
  [ 46  60  88]
  ...
  [115 125 131]
  [115 125 131]
  [116 126 132]]

 [[ 43  56  82]
  [ 44  59  84]
  [ 46  60  88]
  ...
  [115 125 131]
  [116 126 132]
  [116 126 132]]

 [[ 43  56  82]
  [ 44  59  84]
  [ 46  60  88]
  ...
  [115 125 131]
  [116 126 132]
  [116 126 132]]]


NameError: name 'bucket' is not defined

## install + import requirements

In [8]:
import pyaudio

In [ ]:
def audio_transcription():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Say something!")
        while True:
            audio = recognizer.listen(source)
            try:
                # Transcribe the audio to text
                text = recognizer.recognize_google(audio)
                print("Transcription: " + text)
            except sr.UnknownValueError:
                print("Could not understand audio")
            except sr.RequestError as e:
                print("Error; {0}".format(e))

In [9]:
def audio_transcription():
    recognizer = sr.Recognizer()
    output = ""
    with sr.Microphone() as source:
        # Adjust for ambient noise once at the beginning
        recognizer.adjust_for_ambient_noise(source)

        print("Say something!")
        while True:
            try:
                audio = recognizer.listen(source, phrase_time_limit=5)

                # Transcribe the audio to text
                text = recognizer.recognize_google(audio)
                print(text)
                output += text

            except sr.UnknownValueError:
                print("")
            except sr.RequestError as e:
                print("Error; {0}".format(e))
            except sr.WaitTimeoutError:
                print("No speech detected, try again...")
    return output

In [11]:
# Running camera operations in a separate thread
camera_thread = threading.Thread(target=camera_operations)
camera_thread.start()

# Running audio transcription in the main thread
audio_transcription()

# Wait for the camera thread to finish
camera_thread.join()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q4/w5wd298x2j51mk7bkv38kmt80000gn/T/ipykernel_37531/1977272103.py", line 66, in camera_operations
cv2.error: Unknown C++ exception from OpenCV code


: 

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

user = "Rachel"
dialogue = f'''hi nice to meet you what's your name my name is Xander chin
erase my name is Rachel

participating in hack Western 10 by
remember people's names oh what a coincidence I'm doing that'''

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": f'''Extract the name and key information about the person {user} is talking to in relation to {user} from this conversation dialogue:

{dialogue}

This is going to be displayed to {user} as a reminder to them about this person. Use this format and fill in the blanks, if name or location info is not available, assign it [Unknown].:
Name: [insert name]
Where they are: [insert location]
Points:
- [insert point 1]
- [insert point 2]
- [insert point 3]...
'''
    },
    {
      "role": "user",
      "content": ""
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [76]:
print(response.choices[0].message.content)

Name: Xander Chin
Where they are: [Unknown]
Points:
- Participating in hack Western 10
- Working on remembering people's names.
